In [11]:
import time, csv, requests, datetime, random, datetime, os, shutil, glob
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import tzinfo, timedelta, datetime, date
from os import path
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException,TimeoutException,WebDriverException

In [12]:
path = os.path.join(os.getcwd(),"Data")
stockspath = os.path.join(path,"Stock")
corporateactionspath = os.path.join(path,"Corporate Actions")
print(stockspath)
print(corporateactionspath)
print(path)

C:\Users\venu\Desktop\Stock Market Analysis\Data\Stock
C:\Users\venu\Desktop\Stock Market Analysis\Data\Corporate Actions
C:\Users\venu\Desktop\Stock Market Analysis\Data


# Bonus Issue

In [13]:
def bonus_issue(stock,start_date,end_date,r1,r2):
    specific_dates = stock[stock.Date.between(end_date,start_date)]
    for index,row in specific_dates.iterrows():
        specific_dates.loc[index,"Open Price"] = specific_dates.loc[index,"Open Price"] * (r2/(r1+r2))
        specific_dates.loc[index,"Low Price"] = specific_dates.loc[index,"Low Price"] * (r2/(r1+r2))
        specific_dates.loc[index,"High Price"] = specific_dates.loc[index,"High Price"] * (r2/(r1+r2))
        specific_dates.loc[index,"Close Price"] = specific_dates.loc[index,"Close Price"] * (r2/(r1+r2))
        specific_dates.loc[index,"WAP"] = specific_dates.loc[index,"WAP"] * (r2/(r1+r2))
        stock.loc[index] = specific_dates.loc[index]
    return stock

# Stock Split

In [14]:
def stock_split(stock,start_date,end_date,r1,r2):
    specific_dates = stock[stock.Date.between(end_date,start_date)]
    for index,row in specific_dates.iterrows():
        specific_dates.loc[index,"Open Price"] = specific_dates.loc[index,"Open Price"] * (r1/r2)
        specific_dates.loc[index,"Low Price"] = specific_dates.loc[index,"Low Price"] * (r1/r2)
        specific_dates.loc[index,"High Price"] = specific_dates.loc[index,"High Price"] * (r1/r2)
        specific_dates.loc[index,"Close Price"] = specific_dates.loc[index,"Close Price"] * (r1/r2)
        specific_dates.loc[index,"WAP"] = specific_dates.loc[index,"WAP"] * (r1/r2)
        stock.loc[index] = specific_dates.loc[index]
    return stock

# Applying Corporate Actions

In [15]:
def apply_corporate_actions(stock,corporate):
    stock["Date"] = pd.to_datetime(stock["Date"])
    corporate["Ex Date"] = pd.to_datetime(corporate["Ex Date"],errors='coerce')
    # corporate["BC Start Date"] = pd.to_datetime(corporate["BC Start Date"],errors='coerce')
    # corporate[" BC End Date\t"] = pd.to_datetime(corporate[" BC End Date\t"],errors='coerce')
    # corporate["ND Start Date"] = pd.to_datetime(corporate["ND Start Date"],errors='coerce')
    # corporate["ND End Date"] = pd.to_datetime(corporate["ND End Date"],errors='coerce')
    
    bonus_df = corporate[corporate['Purpose'].str.contains("Bonus")]
    for index,row in bonus_df.iterrows():
        start_date = bonus_df.loc[index,"Ex Date"]
        ratio = bonus_df.loc[index,"Purpose"]
        r1,r2 = re.findall(r"\d+",ratio)
        r1,r2 = int(r1),int(r2)
        end_date = stock.tail(1)["Date"].values[0]
        stock = bonus_issue(stock,start_date,end_date,r1,r2)

    stock_split_df = corporate[corporate['Purpose'].str.contains("Stock")]
    for index,row in stock_split_df.iterrows():
        start_date = stock_split_df.loc[index,"Ex Date"]
        ratio = stock_split_df.loc[index,"Purpose"]
        r1,r2 = re.findall(r"\d+",ratio)
        r1,r2 = int(r1),int(r2)
        end_date = stock.tail(1)["Date"].values[0]
        stock = stock_split(stock,start_date,end_date,r1,r2)
    return stock